<a href="https://colab.research.google.com/github/nilu-ai/Restaurant_Fttx_project/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 7.8 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117029 sha256=8ffbda7d6df78005ef7dc1ef6744eeaa1b43efbba5c474f4ae13669878d4d831
  Stored in directory: /root/.cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8859201361e9e581ced8c75e5c6a3
Successfully built fire


In [ ]:
!unzip trainss

Archive:  trainss.zip
   creating: trainss/abhi/
  inflating: trainss/abhi/WIN_20240524_15_28_37_Pro.jpg  
  inflating: trainss/abhi/WIN_20240524_15_28_39_Pro.jpg  
  inflating: trainss/abhi/WIN_20240524_15_28_41_Pro.jpg  
  inflating: trainss/abhi/WIN_20240524_15_28_42_Pro.jpg  
  inflating: trainss/abhi/WIN_20240524_15_28_44_Pro.jpg  
  inflating: trainss/abhi/WIN_20240524_15_28_45_Pro.jpg  
  inflating: trainss/abhi/WIN_20240524_15_28_47_Pro.jpg  
  inflating: trainss/abhi/WIN_20240524_15_28_48_Pro.jpg  
  inflating: trainss/abhi/WIN_20240524_15_28_50_Pro.jpg  
  inflating: trainss/abhi/WIN_20240524_15_28_52_Pro.jpg  
   creating: trainss/Abhishek 1/
  inflating: trainss/Abhishek 1/WIN_20240524_15_28_24_Pro (2).jpg  
  inflating: trainss/Abhishek 1/WIN_20240524_15_28_24_Pro.jpg  
  inflating: trainss/Abhishek 1/WIN_20240524_15_28_25_Pro (2).jpg  
  inflating: trainss/Abhishek 1/WIN_20240524_15_28_25_Pro.jpg  
  inflating: trainss/Abhishek 1/WIN_20240524_15_29_22_Pro.jpg  
  inflatin

In [ ]:
from deepface import DeepFace
df = DeepFace.find(img_path = "/content/trainss/Abhishek 1/WIN_20240524_15_29_24_Pro.jpg", db_path = "/content/trainss",model_name="Facenet512")
print(df)
if df[0].distance.empty:
  print("Unkonw")
elif df[0].distance[0]<0.3:
  print(df)
else:
  print("Unknow User")



24-05-31 11:38:26 - Searching /content/trainss/Abhishek 1/WIN_20240524_15_29_24_Pro.jpg in 36 length datastore
24-05-31 11:38:28 - find function duration 1.9199469089508057 seconds
[                                             identity  \
0   /content/trainss/Abhishek 1/WIN_20240524_15_29...   
1   /content/trainss/Abhishek 1/WIN_20240524_15_28...   
2   /content/trainss/Abhishek 1/WIN_20240524_15_28...   
3   /content/trainss/Abhishek 1/WIN_20240524_15_29...   
4   /content/trainss/Abhishek 1/WIN_20240524_15_29...   
5   /content/trainss/Abhishek 1/WIN_20240524_15_29...   
6   /content/trainss/Abhishek 1/WIN_20240524_15_29...   
7   /content/trainss/Abhishek 1/WIN_20240524_15_28...   
8   /content/trainss/Abhishek 1/WIN_20240524_15_29...   
9   /content/trainss/Abhishek 1/WIN_20240524_15_29...   
10  /content/trainss/Abhishek 1/WIN_20240524_15_29...   
11  /content/trainss/Abhishek 1/WIN_20240524_15_29...   

                                        hash  target_x  target_y  target_w  

In [ ]:
from flask import Flask, request, jsonify
import cv2
import numpy as np
import base64
from deepface import DeepFace
import os
app = Flask(__name__)
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))
@app.route('/', methods=['GET', 'POST'])
def upload_image():
    if request.method == 'POST':
        data = request.json
        img_data = data['image'].split(',')[1]
        nparr = np.frombuffer(base64.b64decode(img_data), np.uint8)
        img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

        try:

            predictions = DeepFace.find(img_path=img, db_path="/content/trainss", model_name="Facenet512")

            if not predictions[0].empty and predictions[0].distance[0] < 0.3:
                response = {
                    'status': 'success',
                    'data': predictions[0].to_dict()
                }
            else:
                response = {
                    'status': 'unknown User'
                }
        except Exception as e:
            response = {
                'status': 'error',
                'message': str(e)
            }

        return jsonify(response)

    return """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
    </head>
    <body>
        <video id="video" width="640" height="480" autoplay></video>
        <canvas id="canvas" width="640" height="480" style="display:none;"></canvas>
        <div id="prediction"></div>
        <script>
            const video = document.getElementById('video');
            const canvas = document.getElementById('canvas');
            const predictionDiv = document.getElementById('prediction');

            async function initCamera() {
                try {
                    const stream = await navigator.mediaDevices.getUserMedia({ video: true });
                    video.srcObject = stream;
                } catch (err) {
                    console.error('Error accessing camera:', err);
                }
            }

            async function captureAndPredict() {
                const context = canvas.getContext('2d');
                context.drawImage(video, 0, 0, canvas.width, canvas.height);
                const imageData = canvas.toDataURL('image/jpeg');
                const response = await fetch('/', {
                    method: 'POST',
                    body: JSON.stringify({ image: imageData }),
                    headers: {
                        'Content-Type': 'application/json'
                    }
                });
                const result = await response.json();
                predictionDiv.innerHTML = JSON.stringify(result, null, 2);
            }

            setInterval(captureAndPredict, 1000);

            initCamera();
        </script>
    </body>
    </html>
    """


@app.route('/new_user', methods=['GET', 'POST'])
def newuser():
    if request.method == 'POST':
        data = request.get_json()
        username = data['username']
        photos = data['photos']

        # Create directory for the new user
        user_dir = os.path.join('trainss', username)
        os.makedirs(user_dir, exist_ok=True)

        # Save photos
        for i, photo in enumerate(photos):
            photo_data = base64.b64decode(photo.split(',')[1])
            with open(os.path.join(user_dir, f'photo_{i+1}.jpg'), 'wb') as f:
                f.write(photo_data)

        return jsonify({'status': 'success', 'message': 'User data saved.'})

    else:
        return '''
            <!DOCTYPE html>
            <html lang="en">
            <head>
                <meta charset="UTF-8">
                <meta name="viewport" content="width=device-width, initial-scale=1.0">
            </head>
            <body>
                <video id="video" width="640" height="480" autoplay></video>
                <canvas id="canvas" width="640" height="480" style="display:none;"></canvas>
                    <label>Name: <input id="username" type="text"/></label>
                <button id="submitimage">Submit New Data</button>
                <script>
                    const video = document.getElementById('video');
                    const canvas = document.getElementById('canvas');
                    const submit = document.getElementById('submitimage');
                    const usernameInput = document.getElementById('username');
                    let photos = [];
                    let photoCount = 0;

                    async function initCamera() {
                        try {
                            const stream = await navigator.mediaDevices.getUserMedia({ video: true });
                            video.srcObject = stream;
                        } catch (err) {
                            console.error('Error accessing camera:', err);
                        }
                    }

                    function capturePhoto() {
                        const context = canvas.getContext('2d');
                        context.drawImage(video, 0, 0, canvas.width, canvas.height);
                        const imageData = canvas.toDataURL('image/jpeg');
                        photos.push(imageData);
                    }

                    async function submitData() {
                        if (photoCount < 3) {
                            capturePhoto();
                            photoCount++;
                            setTimeout(submitData, 1000); // Capture a photo every second
                        } else {
                            const username = usernameInput.value;
                            const response = await fetch('/new_user', {
                                method: 'POST',
                                body: JSON.stringify({ username, photos }),
                                headers: {
                                    'Content-Type': 'application/json'
                                }
                            });
                            const result = await response.json();
                            console.log(result);
                        }
                    }

                    submit.addEventListener('click', () => {
                        photoCount = 0;
                        photos = [];
                        submitData();
                    });

                    initCamera();
                </script>
            </body>
            </html>
        '''



if __name__ == '__main__':
    app.run(host='0.0.0.0')


https://svbiubpeh6b-496ff2e9c6d22116-5000-colab.googleusercontent.com/
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [31/May/2024 12:16:10] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/May/2024 12:16:12] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [31/May/2024 12:16:13] "POST / HTTP/1.1" 200 -


24-05-31 12:16:13 - Searching [[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]] in 36 length datastore
24-05-31 12:16:14 - Searching [[[126 145 148]
  [127 146 149]
  [127 148 150]
  ...
  [157 171 170]
  [156 170 169]
  [157 171 170]]

 [[126 145 148]
  [127 146 149]
  [127 148 150]
  ...
  [158 172 171]
  [158 172 171]
  [160 174 173]]

 [[127 146 149]
  [127 146 149]
  [127 148 150]
  ...
  [160 172 172]
  [161 173 173]
  [162 174 174]]

 ...

 [[159 146 120]
  [161 150 123]
  [177 166 139]
  ...
  [178 202 202]
  [173 197 197]
  [168 192 192]]

 [[157 146 118]
  [165 156 129]
  [180 170 146]
  ...
  [182 206 206]
  [176 200 

INFO:werkzeug:127.0.0.1 - - [31/May/2024 12:16:14] "GET /new_user HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/May/2024 12:16:15] "GET /favicon.ico HTTP/1.1" 404 -


24-05-31 12:16:15 - Searching [[[125 150 154]
  [124 149 153]
  [127 149 154]
  ...
  [161 174 176]
  [162 174 174]
  [163 175 175]]

 [[127 152 156]
  [126 151 155]
  [128 150 155]
  ...
  [161 174 176]
  [163 175 175]
  [163 175 175]]

 [[128 150 155]
  [128 150 155]
  [128 150 155]
  ...
  [161 174 176]
  [163 175 175]
  [164 176 176]]

 ...

 [[160 148 130]
  [165 153 135]
  [179 167 149]
  ...
  [182 204 209]
  [176 198 203]
  [174 196 201]]

 [[153 143 126]
  [163 153 136]
  [178 170 153]
  ...
  [187 209 214]
  [176 198 203]
  [178 200 205]]

 [[152 143 129]
  [166 157 143]
  [185 178 161]
  ...
  [194 216 221]
  [177 199 204]
  [178 200 205]]] in 36 length datastore


...

24-05-31 12:20:43 - Searching [[[185 191 196]
  [185 191 196]
  [185 191 196]
  ...
  [203 211 210]
  [204 212 211]
  [204 212 211]]

 [[185 191 196]
  [185 191 196]
  [185 191 196]
  ...
  [203 211 210]
  [204 212 211]
  [204 212 211]]

 [[185 191 196]
  [185 191 196]
  [185 191 196]
  ...
  [203 211 210]
  [204 212 211]
  [204 212 211]]

 ...

 [[221 218 213]
  [220 220 214]
  [217 217 211]
  ...
  [192 200 190]
  [216 223 216]
  [210 217 210]]

 [[219 218 214]
  [220 219 215]
  [218 217 213]
  ...
  [190 197 190]
  [212 219 212]
  [212 219 212]]

 [[219 218 214]
  [220 219 215]
  [216 217 213]
  ...
  [188 195 188]
  [208 215 208]
  [211 218 211]]] in 36 length datastore
24-05-31 12:20:44 - Searching [[[184 190 197]
  [184 190 197]
  [184 190 195]
  ...
  [203 210 213]
  [206 211 214]
  [206 211 214]]

 [[184 190 197]
  [184 190 197]
  [184 190 195]
  ...
  [203 210 213]
  [205 210 213]
  [206 211 214]]

 [[184 190 197]
  [184 190 197]
  [184 190 195]
  ...
  [205 210 213]
  [205 21

INFO:werkzeug:127.0.0.1 - - [31/May/2024 12:20:45] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/May/2024 12:20:45] "POST / HTTP/1.1" 200 -


24-05-31 12:20:45 - find function duration 0.5647594928741455 seconds
24-05-31 12:20:45 - Searching [[[182 192 186]
  [183 193 187]
  [184 194 188]
  ...
  [203 211 211]
  [203 211 211]
  [203 211 211]]

 [[182 192 186]
  [183 193 187]
  [183 193 187]
  ...
  [203 211 211]
  [203 211 211]
  [203 211 211]]

 [[184 193 190]
  [184 193 190]
  [184 193 190]
  ...
  [203 211 211]
  [202 210 210]
  [202 210 210]]

 ...

 [[216 220 209]
  [215 219 208]
  [215 218 209]
  ...
  [189 195 184]
  [217 220 211]
  [218 221 212]]

 [[214 220 209]
  [213 219 208]
  [214 219 210]
  ...
  [184 190 179]
  [217 220 211]
  [218 221 212]]

 [[215 221 210]
  [214 220 209]
  [213 221 211]
  ...
  [184 190 179]
  [216 219 210]
  [218 221 212]]] in 36 length datastore


INFO:werkzeug:127.0.0.1 - - [31/May/2024 12:20:46] "POST / HTTP/1.1" 200 -


24-05-31 12:20:46 - Searching [[[183 194 191]
  [182 193 190]
  [181 192 190]
  ...
  [197 206 209]
  [197 206 209]
  [200 209 212]]

 [[181 192 189]
  [180 191 188]
  [178 189 187]
  ...
  [197 206 209]
  [197 206 209]
  [199 208 211]]

 [[182 193 191]
  [180 191 189]
  [179 189 189]
  ...
  [197 206 209]
  [197 206 209]
  [199 208 211]]

 ...

 [[215 219 208]
  [214 218 207]
  [214 217 208]
  ...
  [193 199 188]
  [213 219 208]
  [212 218 207]]

 [[215 218 209]
  [214 217 208]
  [214 217 208]
  ...
  [194 200 189]
  [210 216 205]
  [212 218 207]]

 [[215 218 209]
  [214 217 208]
  [214 217 208]
  ...
  [196 202 191]
  [205 211 200]
  [209 215 204]]] in 36 length datastore
